## Authors: Hyewon H. and Alex C.

## Importing packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
from numpy.fft import fft
import music21 as m21
from music21 import *
import math

## The Markov chain

In [2]:
# Define states
LOW_MIDI = 47
HIGH_MIDI = 74
STATES_PER_NOTE = 5
STATE_VALUES =  145 # REPLACE IT WITH THE TOTAL NUMBER OF POSSIBLE STATES (INCLUDING "SILENCE")

# Define initial distribution
init_dist = np.zeros(STATE_VALUES)
for i in range(STATE_VALUES):
    if i % STATES_PER_NOTE  == 0: # equal chance of being in the first state of any note
        init_dist[i] = 1/STATE_VALUES

# Define transition matrix
A = np.zeros((STATE_VALUES, STATE_VALUES))
for i in range(A.shape[0]): # for each state
    
    A[i, i] = 1 # every state has a self loop
    if (i+1)%STATES_PER_NOTE  != 0: # if this state is not the fifth state of a note
        A[i, i+1] = 1 # this state has an arrow to the next state (which is within the same note)
    else: # if this state is the fifth state of a note
        for j in range(STATE_VALUES // STATES_PER_NOTE):
            A[i, j*STATES_PER_NOTE ] = 1

# Normalize every row so each row sums to 1
for i in range(A.shape[0]):
    A[i, ] = A[i, ] / sum(A[i, ])

## Create templates

In [3]:
def peak_fun(x):
    """
    Constructs and returns a harmonic peak
    """
    return math.exp(-0.5*x*x)


def midi2freq(midi):
    """
    Returns the frequency (in Hz) of the given midi number
    """
    return 440*np.power(2, (midi-69)/12)


def template(midi, N, sr):
    """
    Returns a frequency profile template for the given midi note.
        midi: midi number
        N: frame size (must be an even number)
        sr: sample rate
    """
    if N % 2 != 0:
        print("Frame size is not even!")
        return None
    f0 = midi2freq(midi) # fundamental frequency
    H = 13 # maximumn number of harmonics to include
    bins = math.floor(N/2) # half of the DFT result
    template = np.zeros(bins) 
    
    for h in range(1, H+1): # h = 1, 2, 3, ..., H
        bin = round(f0*h*N/sr) # bin index of this harmonic
        if bin >= bins: # exceeds Nyquist frequency
            break
        sigma = bin * 0.01 + 0.01 # controls the width of the peak
        for k in range(bins):
            x = (k-bin)/sigma
            template[k] += math.exp(-k*0.01) * peak_fun(x) # descreasing amplitudes for higher frequencies
    
    for k in range(bins):
        template[k] += 0.002
    # Normalize all bins:
    total = sum(template)
    if (total > 0):
        template = template / total
    else:
        print("The template has 0 everywhere!")
    return template


def create_midi_templates(N, sr):
    """
    Returns a dictionary where the key is a midi number (or 0) and
    the value is the template of this midi note.
    (silence is represented by 0)
    """
    templates = {}
    for midi in range(LOW_MIDI, HIGH_MIDI + 1):
        templates[midi] = template(midi, N, sr)
    # Template for "silence":
    templates[0] = np.repeat(1/math.floor(N/2), math.floor(N/2)) # flat template
    return templates

## Calculate likelihoods

In [4]:
def calculate_likelihood(tmplt, spectrum):
    """
    Calculates and returns the likelihood of observing the spectrum given the template.
        T: the spectral template
        spectrum: the observed spectrum of a frame
        (T and spectrum must have the same length.)
    """
    if np.sum(spectrum) == 0:
        print("In calculate_likelihood: sum of spectrum is 0!")
    else:
        spectrum = spectrum / np.sum(spectrum) # normalize the signal (ignore volumn)
    if (len(tmplt) != len(spectrum)):
        print("The length of the spectrum and the template doesn't match!")
        return None
    log_like = 0
    for k in range(len(tmplt)):
        log_like += (spectrum[k]*math.log(tmplt[k]))
    return math.exp(log_like) # log likelihood -> likelihood


def calculate_all_likelihoods(templates, spectrogram):
    """
    Calculates and returns all the likelihoods of observing each frame in the spectogram
    given each template. The returned value is a two-dimensional array. The row indice represent
    midi numbers, and column indices are the frame indices. Note that Row 1 ~ Row (LOW_MIDI-1)
    are all zeros because they are not considered in our program.
        templates: a dictionary containing the spectral templates of all possible notes (including silence)
        spectrogram: the observed spectrogram including all frames (dimension: bins rows * frames columns)
        (The length of the templates must be the same as the numbers of rows in spectrogram.)
    """
    T = spectrogram.shape[1] # the total number of frames
    likelihoods = np.zeros((HIGH_MIDI+1, T))
    for t in range(T):
        spectrum = spectrogram[:, t]
        likelihoods[0, t] = calculate_likelihood(templates[0], spectrum) # REPLACE 0 WITH THE REAL LIKELIHOOD FOR THE SILENCE NOTE
        for midi in range(LOW_MIDI, HIGH_MIDI + 1):
            likelihoods[midi, t] = calculate_likelihood(templates[midi], spectrum)
    return likelihoods

def xml_to_stream(xml):
    """Convert a music xml file (represented by music21.stream.Score) to a list of note events
    Based on Müller, Meinard (2015), Fundamentals of Music Processing, and adapted by Yucong Jiang.
    Args:
        xml (music21.stream.Score): a music21.stream.Score object
    Returns:
        score (list): A list of note events where each note is specified as
            ``[start, duration, pitch]``
    """
    if isinstance(xml, m21.stream.Score):
        xml_data = xml
    else:
        raise RuntimeError('midi must be a path to a midi file or music21.stream.Score')

    myStream = stream.Stream()
    for part in xml_data.parts:
        instrument = part.getInstrument().instrumentName
        for note in part.flat.notes: # replace flat with flatten() if your music21 is at least v.7
             myStream.append(note)
    return myStream



## Ask for user input: either an audio file (.wav) or a musescore file (.xml)

In [9]:
SR = 8000 # DON'T CHANGE
N = 1024 # DON'T CHANGE
hop = int(N/2) # DON'T CHANGE

original_File = input("What is your original Musescore file name to test against?")
y = converter.parse(original_File)
original_notes = xml_to_stream(y)

#can add a try except block here 
choice = input("Do you want to upload another musescore File (.xml) - 1, or an audio file - 2?")
if choice == "1":
    filePath = input("What is the path to your musescore File?")
    s = converter.parse(filePath)
    notesToTest = xml_to_stream(s)
    
elif choice == "2":
    filePath = input("What is the path to your audio file?")
    samples, sr = librosa.load(filePath, sr = SR)

    # Calculate STFT
    T = math.floor( (len(samples)-N) / hop ) + 1 # the total number of frames
    STFT = np.zeros((N, T), dtype='complex') # the STFT result is a N by T matrix
    hann = 0.5 * ( 1 - np.cos(2*np.pi*np.arange(N)/N) ) # Hann window
    for t in range(T): # for each frame
        chunck = samples[t*hop : t*hop + N] # get the frame t of audio data
        X = fft(chunck * hann) # apply FFT on windowed signal (the result contains N complex numbers)
        STFT[:, t] = X
    spect = np.abs(STFT[:int(N/2), :]) # complex numbers --> abs values

    # Calculate likelihoods for each midi (NOT state) at each frame (row 0 represents silence)
    midi_templates = create_midi_templates(N, sr)
    likelihoods = calculate_all_likelihoods(midi_templates, spect)

    optimal_score = np.zeros((STATE_VALUES, T))
    optimal_pred = np.zeros((STATE_VALUES, T), dtype = int)

    for s in range(STATE_VALUES):
        if s < 5:
            optimal_score[s, 0] = init_dist[s] * likelihoods[0, 0]
        else:
            optimal_score[s, 0] = init_dist[s] * likelihoods[s//5 + LOW_MIDI-1, 0]
    for t in range(T):
        for s in range(STATE_VALUES):
            for p in range(STATE_VALUES):
                if s < 5:
                    score = optimal_score[p, t-1] * A[p,s] * likelihoods[0, t]
                else:
                    score = optimal_score[p, t-1] * A[p,s] * likelihoods[s//5 + LOW_MIDI -1, t]
                if score > optimal_score[s,t]:
                    optimal_score[s,t] = score
                    optimal_pred[s,t] = p
        optimal_score[:, t] /= sum(optimal_score[:, t])

    result = np.zeros(T, dtype=int) # stores the inference result
    result[T-1] = np.argmax(optimal_score[:, T-1]) # choose the best final state
    # Follow pointers back through trellis:
    for t in range(T-2, -1, -1): # t = 98, 97, ..., 1, 0
        result[t] = optimal_pred[result[t+1], t+1]
    # result = a numpy array that stores the most likely state sequence

    notes = (result // STATES_PER_NOTE ).astype(float)
    notes[notes==0] = np.nan # set every 0 to be nan to avoid plotting silence (midi=0)
    recognized_midi = LOW_MIDI - 1 + notes
    
    wholeNote = duration.Duration('whole')
    totalLength = len(recognized_midi)
    noteLength = totalLength//8 #this will change if you have more than eight notes
    notesToTest = stream.Stream()
    for i in range(8): 
        currNote = note.Note(pitch.Pitch(recognized_midi[33*i]))
        notesToTest.append(currNote)
else:
    print("invalid choice, try again")

What is your original Musescore file name to test against?C Major.xml
Do you want to upload another musescore File (.xml) - 1, or an audio file - 2?1
What is the path to your musescore File?Test 1.xml


## Display results

In [10]:
msg = "For the notes: "

for i in range(len(original_notes)):
    msg += str(original_notes[i].pitch) + " "
print(msg)

msg1 = "You played: "
for i in range(len(notesToTest)):
    msg1 += str(notesToTest[i].pitch) + " "
print(msg1)

msg2 = "Making your played harmonies: \n"
for i in range(len(original_notes)):
    ch = chord.Chord([str(original_notes[i].pitch), str(notesToTest[i].pitch)])
    msg2 += "In measure " + str(i+1) + ": " + str(ch.commonName) + " \n"
print(msg2)

For the notes: C4 D4 E4 F4 G4 A4 B4 C5 
You played: G4 G4 B4 C#5 D4 A-4 F4 D4 
Making your played harmonies: 
In measure 1: Perfect Fifth 
In measure 2: Perfect Fourth 
In measure 3: Perfect Fifth 
In measure 4: Augmented Fifth 
In measure 5: Perfect Fourth 
In measure 6: Diminished Unison 
In measure 7: Augmented Fourth 
In measure 8: Minor Seventh 

